# MoodSync
#### Realtime Emotion Monitoring with Deep Learning (With Camera)

In [3]:
# Import the 'cv2' library for computer vision operations.
import cv2

# Import the 'model_from_json' function from Keras for loading a model from JSON.
from keras.models import model_from_json

# Import the 'numpy' library for numerical operations.
import numpy as np


In [4]:
# Open the JSON file containing the model architecture for reading.
json_file = open("facialemotionmodel.json", "r")

# Read the contents of the JSON file.
model_json = json_file.read()

# Close the JSON file.
json_file.close()

# Create a new model from the loaded architecture.
model = model_from_json(model_json)


In [5]:
# Load the model's weights from an HDF5 file.
model.load_weights("facialemotionmodel.h5")

# Define the path to the Haar Cascade XML file for face detection.
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'

# Create a face cascade classifier using the Haar Cascade XML file.
face_cascade = cv2.CascadeClassifier(haar_file)


### Define a function to extract features from an image.

In [6]:
def extract_features(image):
    feature = np.array(image)  # Convert the image to a NumPy array.
    feature = feature.reshape(1, 48, 48, 1)  # Reshape the feature.
    return feature / 255.0  # Normalize the pixel values.


### Open the webcam for video capture.

In [7]:
webcam = cv2.VideoCapture(0)

# Define a dictionary to map class indices to emotion labels.
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Main loop for real-time emotion recognition from webcam feed.
while True:
    # Read a frame from the webcam.
    i, im = webcam.read()

    # Convert the frame to grayscale for face detection.
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame using Haar Cascade classifier.
    faces = face_cascade.detectMultiScale(im, 1.3, 5)

    try:
        # Process each detected face.
        for (p, q, r, s) in faces:
            # Extract the face region and draw a rectangle around it.
            image = gray[q:q+s, p:p+r]
            cv2.rectangle(im, (p, q), (p+r, q+s), (255, 0, 0), 2)

            # Resize the face region to match the model's input size.
            image = cv2.resize(image, (48, 48))

            # Extract features from the resized image.
            img = extract_features(image)

            # Use the trained model to predict the emotion label.
            pred = model.predict(img)
            prediction_label = labels[pred.argmax()]

            # Overlay the predicted emotion label on the frame.
            cv2.putText(im, '% s' % (prediction_label), (p-10, q-10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 255))

        # Display the frame with face detection and emotion recognition results.
        cv2.imshow("Output", im)

        # Wait for a key press, exit the loop if 'Esc' is pressed.
        cv2.waitKey(27)

    except cv2.error:
        # Handle errors (e.g., when no faces are detected).
        pass


1/1 [==============================] - 0s 194ms/step


1/1 [==============================] - 1s 504ms/step
